# Scraping BestFightOdds.com

In [1]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     
import shutil      
import datetime
from scipy.stats import norm
import xgboost
from xgboost import XGBClassifier
from random import randint
import  random
import os
os.chdir('C:/Users/Travis/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

c:\Users\Travis\anaconda3\envs\Trav310\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Excel-fixed data:

In [60]:
fight_odds_urls = pd.read_csv('data/final/events/Final_Hand_Done_BFO_Urls_with UFC_Stats_Urls.csv')

print(fight_odds_urls.shape)
fight_odds_urls.head(3)

(598, 13)


,event_title,event_url,odds_search,event_odds_url,Short_title,Short_Title_2 (vs. to vs),Name1,Name2,Check_Name1,Check_Name1.1,Trav_Check,GOOGLE SEARCHER,Googlez
0,UFC Fight Night: Thompson vs. Holland,http://ufcstats.com/event-details/b23388ff8ac6...,https://www.bestfightodds.com/search?query=UFC...,https://www.bestfightodds.com/events/ufc-fight...,Thompson vs. Holland,Thompson vs Holland,thompson,holland,False,False,ok,https://www.google.com/search?q=site:bestfight...,https://www.google.com/search?q=site:bestfight...
1,UFC Fight Night: Nzechukwu vs. Cutelaba,http://ufcstats.com/event-details/012fc7cd0779...,https://www.bestfightodds.com/search?query=UFC...,https://www.bestfightodds.com/events/ufc-fight...,Nzechukwu vs. Cutelaba,Nzechukwu vs Cutelaba,nzechukwu,cutelaba,False,False,ok,https://www.google.com/search?q=site:bestfight...,https://www.google.com/search?q=site:bestfight...
2,UFC 281: Adesanya vs. Pereira,http://ufcstats.com/event-details/b3b6e80b7d5f...,https://www.bestfightodds.com/search?query=UFC...,https://www.bestfightodds.com/events/ufc-281-2529,Adesanya vs. Pereira,Adesanya vs Pereira,adesanya,pereira,False,False,ok,https://www.google.com/search?q=site:bestfight...,https://www.google.com/search?q=site:bestfight...


In [8]:
# get all event urls
event_urls = fight_odds_urls['event_url'].unique()
print(f' {len(event_urls)} events')

 509 events


#### Scraping BFO via Requests & Headers

In [5]:
# you can scrape bestfightodds by pretending to be a browser
# you will get a 404 error if you don't do this
# EXAMPLE: 

url = 'https://www.bestfightodds.com/events/ufc-282-2621'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
r = requests.get(url, headers = header)
dfs = pd.read_html(r.text)
dfs[0]

,Jan Brachowicz
0,Magomed Ankalaev
1,Over 1½ rounds
2,Under 1½ rounds
3,Over 2½ rounds
4,Under 2½ rounds
...,...
1725,Under 7½ fights go the distance
1726,Over 8½ fights go the distance
1727,Under 8½ fights go the distance
1728,Over 9½ fights go the distance


## Scraping BFO Data

In [19]:
fight_odds_urls = pd.read_csv('data/final/events/Final_Hand_Done_BFO_Urls_with UFC_Stats_Urls.csv')
fight_odds_urls.head(5)

,event_title,event_url,odds_search,event_odds_url,Short_title,Short_Title_2 (vs. to vs),Name1,Name2,Check_Name1,Check_Name1.1,Trav_Check,GOOGLE SEARCHER,Googlez
0,UFC Fight Night: Thompson vs. Holland,http://ufcstats.com/event-details/b23388ff8ac6...,https://www.bestfightodds.com/search?query=UFC...,https://www.bestfightodds.com/events/ufc-fight...,Thompson vs. Holland,Thompson vs Holland,thompson,holland,False,False,ok,https://www.google.com/search?q=site:bestfight...,https://www.google.com/search?q=site:bestfight...
1,UFC Fight Night: Nzechukwu vs. Cutelaba,http://ufcstats.com/event-details/012fc7cd0779...,https://www.bestfightodds.com/search?query=UFC...,https://www.bestfightodds.com/events/ufc-fight...,Nzechukwu vs. Cutelaba,Nzechukwu vs Cutelaba,nzechukwu,cutelaba,False,False,ok,https://www.google.com/search?q=site:bestfight...,https://www.google.com/search?q=site:bestfight...
2,UFC 281: Adesanya vs. Pereira,http://ufcstats.com/event-details/b3b6e80b7d5f...,https://www.bestfightodds.com/search?query=UFC...,https://www.bestfightodds.com/events/ufc-281-2529,Adesanya vs. Pereira,Adesanya vs Pereira,adesanya,pereira,False,False,ok,https://www.google.com/search?q=site:bestfight...,https://www.google.com/search?q=site:bestfight...
3,UFC Fight Night: Rodriguez vs. Lemos,http://ufcstats.com/event-details/756f45905fb2...,https://www.bestfightodds.com/search?query=UFC...,https://www.bestfightodds.com/events/ufc-fight...,Rodriguez vs. Lemos,Rodriguez vs Lemos,rodriguez,lemos,False,False,ok,https://www.google.com/search?q=site:bestfight...,https://www.google.com/search?q=site:bestfight...
4,UFC Fight Night: Kattar vs. Allen,http://ufcstats.com/event-details/be5aab761c40...,https://www.bestfightodds.com/search?query=UFC...,https://www.bestfightodds.com/events/ufc-fight...,Kattar vs. Allen,Kattar vs Allen,kattar,allen,False,False,ok,https://www.google.com/search?q=site:bestfight...,https://www.google.com/search?q=site:bestfight...


In [23]:
all_event_odds_urls = fight_odds_urls.event_odds_url.unique()
len(all_event_odds_urls)

490

In [21]:
len('https://www.bestfightodds.com/events/')

37

In [25]:
def get_odds(url):
        # you can scrape by pretending to be a browser
        header = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
        }
        r = requests.get(url, headers = header)
        dfs = pd.read_html(r.text)
        data = dfs[1]
        data.rename(columns={'Unnamed: 0': 'fighter'}, inplace=True)
        data['event_odds_url'] = url
        event_name = url.split('/')[4]
        data['event_id'] = event_name
        data.to_csv('data/final/events/odds_per_event/' + event_name + '.csv', index=False)
        return data



In [24]:
get_odds(all_event_odds_urls[0]).head(2)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,Bet365,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id
0,Kevin Holland,-170▲,-185▼,-195▼,-200▼,-166▼,NaN,-200▼,-175▲,-188▲,NaN,-200▼,NaN,118.0,NaN,https://www.bestfightodds.com/events/ufc-fight...,ufc-fight-night-214-2607
1,Stephen Thompson,+145▼,+150▲,+165▲,+160▲,+140▲,NaN,+160▲,+150▼,+150▼,NaN,+170▲,NaN,118.0,NaN,https://www.bestfightodds.com/events/ufc-fight...,ufc-fight-night-214-2607


In [33]:
# drop nan
all_event_odds_urls = all_event_odds_urls[~pd.isnull(all_event_odds_urls)]
all_event_odds_urls

array(['https://www.bestfightodds.com/events/ufc-fight-night-214-2607',
       'https://www.bestfightodds.com/events/ufc-fight-night-215-2633',
       'https://www.bestfightodds.com/events/ufc-281-2529',
       'https://www.bestfightodds.com/events/ufc-fight-night-214ting-championship-2612',
       'https://www.bestfightodds.com/events/ufc-fight-night-213-2606',
       'https://www.bestfightodds.com/events/ufc-281-2586',
       'https://www.bestfightodds.com/events/ufc-fight-night-212-2579',
       'https://www.bestfightodds.com/events/ufc-vegas-57-2489',
       'https://www.bestfightodds.com/events/ufc-fight-night-211-2569',
       'https://www.bestfightodds.com/events/ufc-279-2541',
       'https://www.bestfightodds.com/events/ufc-fight-night-gane-vs-tuivasa-2518',
       'https://www.bestfightodds.com/events/ufc-278-usman-vs-edwards-2-2545',
       'https://www.bestfightodds.com/events/ufc-fight-night-vera-vs-cruz-2552',
       'https://www.bestfightodds.com/events/ufc-on-espn-santo

In [35]:
n=0
errors = []
complete = []
files = os.listdir('data/final/events/odds_per_event/')
file_list = [n[:-4] for n in files]

for url in all_event_odds_urls:
    url_file_name = url.split('/')[-1]
    url_file_name2 = url_file_name[:-4]
    if url_file_name not in file_list:
        try:
            get_odds(url)
            print(f'{n} / {len(all_event_odds_urls)}')
            n+=1
        except:
            print(f'ERROR! {n} / {len(all_event_odds_urls)}')
            errors.append(url)
            n+=1
    else:
        print(f'{url_file_name} already exists')
        complete.append(url_file_name)

print(f' total errors: {len(errors)}')

ufc-fight-night-214-2607 already exists
ufc-fight-night-215-2633 already exists
ufc-281-2529 already exists
ufc-fight-night-214ting-championship-2612 already exists
ufc-fight-night-213-2606 already exists
ufc-281-2586 already exists
ufc-fight-night-212-2579 already exists
ufc-vegas-57-2489 already exists
ufc-fight-night-211-2569 already exists
ufc-279-2541 already exists
ufc-fight-night-gane-vs-tuivasa-2518 already exists
ufc-278-usman-vs-edwards-2-2545 already exists
ufc-fight-night-vera-vs-cruz-2552 already exists
ufc-on-espn-santos-vs-hill-2534 already exists
ufc-277-pena-vs-nunes-2-2517 already exists
ufc-fight-night-blaydes-vs-aspinall-2519 already exists
ufc-on-abc-ortega-vs-rodriguez-2524 already exists
ufc-on-espn-dos-anjos-vs-fiziev-2501 already exists
ufc-276-adesanya-vs-cannonier-2478 already exists
ufc-on-espn-tsarukyan-vs-gamrot-2574 already exists
ufc-fight-night-kattar-vs-emmett-2523 already exists
ufc-275-teixeira-vs-prochazka-2561 already exists
ufc-fight-night-211-244

### Append Together

In [36]:
#append 
files = os.listdir('data/final/events/odds_per_event/')

In [37]:
all_event_odds = []

for file in files:
    df = pd.read_csv('data/final/events/odds_per_event/' + file)
    # append
    all_event_odds.append(df)

all_event_odds = pd.concat(all_event_odds)

print(all_event_odds.shape)
all_event_odds.head()

(328156, 17)


,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+205▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
3,Thiago Alves,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+250▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
4,Dan Henderson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-200▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN


### Adding Event Names & URLs to Event_Odds df

In [38]:
fight_odds_urls.head(3)

,event_title,event_url,odds_search,event_odds_url,Short_title,Short_Title_2 (vs. to vs),Name1,Name2,Check_Name1,Check_Name1.1,Trav_Check,GOOGLE SEARCHER,Googlez
0,UFC Fight Night: Thompson vs. Holland,http://ufcstats.com/event-details/b23388ff8ac6...,https://www.bestfightodds.com/search?query=UFC...,https://www.bestfightodds.com/events/ufc-fight...,Thompson vs. Holland,Thompson vs Holland,thompson,holland,False,False,ok,https://www.google.com/search?q=site:bestfight...,https://www.google.com/search?q=site:bestfight...
1,UFC Fight Night: Nzechukwu vs. Cutelaba,http://ufcstats.com/event-details/012fc7cd0779...,https://www.bestfightodds.com/search?query=UFC...,https://www.bestfightodds.com/events/ufc-fight...,Nzechukwu vs. Cutelaba,Nzechukwu vs Cutelaba,nzechukwu,cutelaba,False,False,ok,https://www.google.com/search?q=site:bestfight...,https://www.google.com/search?q=site:bestfight...
2,UFC 281: Adesanya vs. Pereira,http://ufcstats.com/event-details/b3b6e80b7d5f...,https://www.bestfightodds.com/search?query=UFC...,https://www.bestfightodds.com/events/ufc-281-2529,Adesanya vs. Pereira,Adesanya vs Pereira,adesanya,pereira,False,False,ok,https://www.google.com/search?q=site:bestfight...,https://www.google.com/search?q=site:bestfight...


In [39]:
def get_event_title(event_odds_url):
    # even[event_name] is the event name
    # even[event_odds_url] is the url
    data = fight_odds_urls[fight_odds_urls['event_odds_url'] == event_odds_url]
    event_name = data.event_title.values[0]
    return event_name
    

In [40]:
def get_ufcstats_url(event_odds_url):
    data = fight_odds_urls[fight_odds_urls['event_odds_url'] == event_odds_url]
    event_url = data.event_url.values[0]
    return event_url

In [41]:
all_event_odds['event_name'] = all_event_odds.apply(lambda row: get_event_title(row['event_odds_url']), axis=1)

In [42]:
all_event_odds['event_ufcstats_url'] = all_event_odds.apply(lambda row: get_ufcstats_url(row['event_odds_url']), axis=1)

In [43]:
all_event_odds.head(3)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365,event_name,event_ufcstats_url
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+205▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...


In [44]:
all_event_odds.to_csv('data/final/odds/All_Odds_by_Fighter_V1.csv', index=False)

In [45]:
all_event_odds['5D'] = all_event_odds['5D'].str[:-1]
all_event_odds['5D'] = all_event_odds['5D'].astype(float)
all_event_odds.head(3)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365,event_name,event_ufcstats_url
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...


In [46]:
all_event_odds['Ref'] = all_event_odds['Ref'].str[:-1]
all_event_odds['Ref'] = all_event_odds['Ref'].astype(float)
all_event_odds.head(3)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365,event_name,event_ufcstats_url
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...


In [47]:
cols = ['fighter', '5D', 'Ref', 'event_odds_url', 'event_ufcstats_url', 'event_id', 'event_name']
all_event_odds = all_event_odds[cols]
all_event_odds.head()

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100


In [48]:
# get shape
all_event_odds.shape

(328156, 7)

In [49]:
all_event_odds.to_csv('data/final/odds/All_Odds_by_Fighter_V2.csv', index=False)


## Summary
- ALL ODDS by fighter are available at:
  - data/ufc_BestFightOdds/All_Event_Odds_by_Fighter.csv
- You can combine this with UfcStatsData easily using 'event_ufcstats_url' 

# Add All Events Df

Saved in final/events

In [50]:
all_event_odds = pd.read_csv('data/final/odds/All_Odds_by_Fighter_V2.csv')

In [51]:
# events are located in data/ufc_stats/events2

folder = 'data/ufc_stats/events2/'

# aggregate all events into one dataframe
all_events = []

for file in os.listdir(folder):
    df = pd.read_csv(folder + file)
    # append
    all_events.append(df)

all_events = pd.concat(all_events)

print(all_events.shape)
all_events.head()

(6912, 19)


,Unnamed: 0,W/L,Weight class,Method,Round,Time,Fighter1,Fighter2,F1_Kd,F2_Kd,F1_Str,F2_Str,F1_Td,F2_Td,F1_Sub,F2_Sub,fight_num,event_id,fight_link
0,0,win,Heavyweight,KO/TKO Punches,3,4:23,Ciryl Gane,Tai Tuivasa,1,1,110,29,0,0,0,0,1,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/810c9da0...
1,1,win,Middleweight,U-DEC,3,5:00,Robert Whittaker,Marvin Vettori,0,0,74,33,1,0,0,0,2,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/b8ca1acd...
2,2,win,Middleweight,U-DEC,3,5:00,Nassourdine Imavov,Joaquin Buckley,0,0,51,46,2,0,2,0,3,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/20f316f9...
3,3,win,Middleweight,KO/TKO Punches,3,1:09,Roman Kopylov,Alessio Di Chirico,1,0,59,39,0,0,0,0,4,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/1a191251...
4,4,win,Featherweight,U-DEC,3,5:00,William Gomis,Jarno Errens,0,0,32,20,3,0,0,1,5,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/398db400...


In [52]:
all_events.to_csv('data/final/events/All_Events_V1.csv', index=False)

In [53]:
all_events = pd.read_csv('data/final/events/All_Events_V1.csv')

# Add Odd Changes to Odds

In [54]:
def get_odds(url):
        # you can scrape by pretending to be a browser
    header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
    }
    r = requests.get(url, headers = header)
    dfs = pd.read_html(r.text)
    data = dfs[1]
    data.rename(columns={'Unnamed: 0': 'fighter'}, inplace=True)
    data['event_odds_url'] = url
    event_name = url.split('/')[4]
    data['event_id'] = event_name
    #data.to_csv('data/ufc_BestFightOdds/odds_by_event/' + event_name + '.csv', index=False)

    return data

In [55]:
def get_bfodds(event_url):
    # scrape the changes in odds by event url
    
    driver = webdriver.Chrome()
    driver.get(event_url)

    # click show all
    driver.find_element_by_class_name('event-swing-expand').click()
    #wait 2 seconds
    time.sleep(2)

    event_swing_container = driver.find_element_by_id('event-swing-container')
    event_swing_container.text


    # the first half of the text is the odds changes, the seconds half are the names. Lets match them up. 
    # first split the text by new line
    event_swing_container.text.split('\n')

    # get the names. You can identify them because they will be more than 5 characters long
    names = [x for x in event_swing_container.text.split('\n') if len(x) > 5]
    names

    # get the odds changes
    odds_changes = [x for x in event_swing_container.text.split('\n') if '%' in x]
    odds_changes

    # get the event name
    event_name = driver.find_element_by_class_name('table-header')
    # name is h1 in event_name
    name = event_name.find_element_by_tag_name('h1')
    event = name.text

    # get the url
    url = driver.current_url


    # create a dataframe
    df = pd.DataFrame({'names': names, 'odds_changes': odds_changes, 'event': event, 'event_url': url})
    
    #close the driver
    driver.close()

    return df

In [56]:
all_odds_eventlist = all_event_odds['event_odds_url'].unique().tolist()

In [105]:
already_complete = []
failed = []
newly_complete = []

for url in all_odds_eventlist:
    event_name = url.split('/')[-1]
    if  str(event_name) + '.csv' in os.listdir('data/final/odds/odds_changes/'):
        print('already done with ' + url.split('/')[-1])
        already_complete.append(event_name)
        continue

    else:
        try:
            df = get_bfodds(url)
            name = url.split('/')[-1]
            df.to_csv('data/final/odds/odds_changes/' + name + '.csv', index=False)
            newly_complete.append(name)
            print('done with ' + name)
        except:
            name = url.split('/')[-1]
            failed.append(name)
            print('failed on ' + name)
            continue
print("")
print('already done with ' + str(len(already_complete)) + ' events')
print('newly done with ' + str(len(newly_complete)) + ' events')
print('failed on ' + str(len(failed)) + ' events')

already done with ufc-100-137
already done with ufc-101-declaration-145
already done with ufc-102-couture-vs-nogueira-156
already done with ufc-103-franklin-vs-belfort-168
already done with ufc-105-couture-vs-vera-201
already done with ufc-106-ortiz-vs-griffin-2-209
already done with ufc-107-penn-vs-sanchez-189
already done with ufc-108-evans-vs-silva-219
already done with ufc-109-relentless-226
already done with ufc-110-nogueira-vs-velasquez-225
already done with ufc-111-st-pierre-vs-hardy-236
already done with ufc-112-invincible-245
already done with ufc-113-machida-vs-shogun-ii-247
already done with ufc-115-liddell-vs-franklin-255
already done with ufc-116-lesnar-vs-carwin-265
already done with ufc-118-edgar-vs-penn-2-278
already done with ufc-119-mir-vs-cro-cop-296
already done with ufc-120-bisping-vs-akiyama-294
already done with ufc-121-lesnar-vs-velasquez-308
already done with ufc-122-marquardt-vs-okami-323
already done with ufc-123-rampage-vs-machida-317
already done with ufc-1

#### Note: 4 Events did not work:

In [106]:
failed

['ufc-236-holloway-vs-poirier-2-1666',
 'ufc-279-2541',
 'ufc-fight-night-138-volkan-vs-smith-1564']

In [107]:
# aggregate all odds changes into one dataframe
all_odds_changes = []

for file in os.listdir('data/final/odds/odds_changes/'):
    df = pd.read_csv('data/final/odds/odds_changes/' + file)
    # append
    all_odds_changes.append(df)

all_odds_changes = pd.concat(all_odds_changes)

print(all_odds_changes.shape)
all_odds_changes

(11323, 4)


,names,odds_changes,event,event_url
0,Deron Winn,-61%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
1,Chuck Liddell,-30%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
2,Gleison Tibau,-28%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
3,Ricardo Palacios,-14%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
4,Kenneth Bergh,+3%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
...,...,...,...,...
19,Daniel Santos,+34%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...
20,Brendan Allen,+35%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...
21,Mike Davis,+55%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...
22,Philipe Lins,+60%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...


In [108]:
all_odds_changes.to_csv('data/final/aggregates/All_Odds_Changes_V1.csv', index=False)

### Edit Event_odds 

Get rid of alternate lines

In [109]:
all_event_odds = pd.read_csv('data/final/odds/All_Odds_by_Fighter_V2.csv')
all_event_odds

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
...,...,...,...,...,...,...,...
328151,Any other result,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
328152,Either fighter wins by unanimous decision,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
328153,Any other result,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
328154,Either fighter wins by split/majority decision,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan


In [110]:
all_options = all_event_odds['fighter'].value_counts()
all_options = pd.DataFrame(all_options)
all_options.reset_index(inplace=True)
delete_options = all_options.head(120)
delete_options

,index,fighter
0,Any other result,90091
1,Fight doesn't go to decision,4614
2,Fight goes to decision,4614
3,Fight is a draw,4608
4,Fight is not a draw,4608
...,...,...
115,Over 4½ rounds,194
116,Exactly one fight goes the distance,180
117,Over 1½ fights go the distance,180
118,Under 1½ fights go the distance,180


In [111]:
delete_options_list = delete_options['index'].tolist()

In [113]:
# delete rows when fighter is in delete_options_list
all_event_odds = all_event_odds[~all_event_odds['fighter'].isin(delete_options_list)]
all_event_odds.head(2)

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100


In [114]:
# delete any rows where fighter has more than 3 words
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('round')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('decision')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('submission')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('knockout')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('draw')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('scorecards')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('majority')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('Not')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('wins')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('losses')]
all_event_odds


,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
...,...,...,...,...,...,...,...
327749,Krzysztof Jotko,NaN,-124.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
327884,Chelsea Chandler,NaN,113.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
327885,Julija Stoliarenko,NaN,-133.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
328020,Guido Cannetti,NaN,250.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan


# Add Odds Change to Odds

In [118]:

def get_odds_change_from_db(event_odds_url, fighter):
    
    # get the odds change from the all_odds_changes dataframe

    try:
        odds_change = all_odds_changes[(all_odds_changes['event_url'] == event_odds_url) & (all_odds_changes['names'] == fighter)]['odds_changes'].values[0]
        return odds_change
    except:
        return np.nan

In [119]:
# test
get_odds_change_from_db('https://www.bestfightodds.com/events/ufc-100-137', 'Brock Lesnar')

'-31%'

In [120]:
all_event_odds['odds_change'] = all_event_odds.apply(lambda row: get_odds_change_from_db(row['event_odds_url'], row['fighter']), axis=1)
all_event_odds

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name,odds_change
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-31%
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,+59%
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,NaN
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-6%
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,+4%
...,...,...,...,...,...,...,...,...
327749,Krzysztof Jotko,NaN,-124.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-32%
327884,Chelsea Chandler,NaN,113.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-6%
327885,Julija Stoliarenko,NaN,-133.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,+4%
328020,Guido Cannetti,NaN,250.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-22%


In [122]:
# get rid of %
all_event_odds['odds_change'] = all_event_odds['odds_change'].str.replace('%', '')
# turn into float
all_event_odds['odds_change'] = all_event_odds['odds_change'].astype(float)
all_event_odds.head(3)

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name,odds_change
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-31.0
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,59.0
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,NaN


In [ ]:
# save to csv
all_event_odds.to_csv('data/final/odds/All_Odds_by_Fighter_WithChange.csv', index=False)